Computing Surprise With ConvoKit
=====================
This notebook provides a demo of how to use the Surprise transformer to compute surprise across a corpus. The transformer currently only allows computation of how surprising a speaker's utterances in one conversation (target) are compared to their utterances in all other conversations (context) in the corpus. Eventually, the functionality of the Surprise transformer will be abstracted to allow for computation of surprise between any target and context types.

In [2]:
import convokit
import numpy as np
from convokit import Corpus, download, Surprise

Step 1: Load a corpus
--------
For now, we will use data from the subreddit r/Cornell to demonstrate the functionality of this transformer

In [3]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at C:\Users\rgang\.convokit\downloads\subreddit-Cornell


In [4]:
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In order to speed up the demo, we will take just the top 100 most active speakers (based on the number of conversations they participate in).

In [5]:
SPEAKER_BLACKLIST = ['[deleted]', 'DeltaBot', 'AutoModerator']
def utterance_is_valid(utterance):
    return utterance.speaker.id not in SPEAKER_BLACKLIST and utterance.text

In [6]:
corpus.organize_speaker_convo_history(utterance_filter=utterance_is_valid)

In [7]:
speaker_activities = corpus.get_attribute_table('speaker', ['n_convos'])

In [8]:
speaker_activities.sort_values('n_convos', ascending=False).head(10)

,n_convos
id,
laveritecestla,781.0
EQUASHNZRKUL,726.0
CornHellUniversity,696.0
t3hasiangod,647.0
ilovemymemesboo,430.0
omgdonerkebab,425.0
cartesiancategory,341.0
cornell256,330.0
mushiettake,321.0


In [9]:
top_speakers = speaker_activities.sort_values('n_convos', ascending=False).head(100).index

In [10]:
import itertools

subset_utts = [list(corpus.get_speaker(speaker).iter_utterances()) for speaker in top_speakers]
subset_corpus = Corpus(utterances=list(itertools.chain(*subset_utts)))

In [11]:
subset_corpus.print_summary_stats()

Number of Speakers: 100
Number of Utterances: 20700
Number of Conversations: 6904


Step 2: Create instance of surprise transformer
---------------
`target_sample_size` and `context_sample_size` specify the minimum number of tokens that should be in the target and context respectively. If the target or context is too short, the transformer will set the surprise to be `nan`. If we sent these to simply be 1, the most surprising statements tend to just be the very short statements. The transformer takes `n_samples` samples from the target and context transformer (where samples are of size corresponding to `target_sample_size` and `context_sample_size`). It calculates cross entropy for each pair of samples and takes the average to get the final surprise score. This is done to minimize effect of length on scores.

In [12]:
surp = Surprise(target_sample_size=100, context_sample_size=100, n_samples=50)

Step 3: Fit transformer to corpus
-----


In [13]:
surp = surp.fit(subset_corpus, group_models_by=['speaker'])

Step 4: Transform corpus
--------
Currently, this transforms each utterance in the corpus adding a field to its metadata with the calculated surprise.

In [14]:
transformed_corpus = surp.transform(subset_corpus, 'utterance', group_target_by=['speaker', 'conversation'], context_selector=lambda s, t: (s.index.get_level_values('speaker') == t[0]) & (s.index.get_level_values('conversation_id') != t[1]), model_selector=lambda ind: ind[0])

Analysis
------
Let's take a look at some of the most surprising speaker conversation involvements.

In [15]:
most_surprising = transformed_corpus.get_utterances_dataframe().sort_values('meta.surprise', ascending=False).head(10)
most_surprising

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,meta.surprise
id,,,,,,,,,,,,,,,
dbrix3s,1483037010,BILL BILL BILL BILL BILL BILL BILL BILL BILL B...,Straight_Derpin,dbqkxgp,5kst5l,22,dbqkxgp,1484130862,0,None,Cornell,False,,CS 2020,5.08975
dbrnysy,1483043255,"Memes aside, I honestly don't know a good answ...",Straight_Derpin,5kst5l,5kst5l,2,dbrnysy,1484133363,0,None,Cornell,False,,CS 2020,5.08975
diujudc,1497369546,Fall 2011 Admissions Stats:\n\nSchool | Apps |...,Dr_Narwhal,6h08sg,6h08sg,9,diujudc,1499312897,0,None,Cornell,False,,Physics &amp; Mathematics 2019,4.56458
cbnqq4d,1376508444,Even better is Tracfone's StraightTalk. They'r...,arandomaltaccount,cbnjwiu,1kbz4p,1,cbnjwiu,1429829700,0,None,Cornell,False,,,4.37768
c5sqkzc,1344908188,Fuck YAF. And fuck you. Don't let that hate ...,omgdonerkebab,y6cd9,y6cd9,0,c5sqkzc,1429631413,0,None,Cornell,False,,,4.34288
d64vv88,1470366925,"Eh, I would be hesitant to wear those. From th...",t3hasiangod,4w8d7e,4w8d7e,5,d64vv88,1473230691,0,None,Cornell,False,,,4.32161
dosp7xl,1508806551,"Far above Cayuga's waters,\nwith its waves of ...",SwissWatchesOnly,dosp7mm,78cb5y,-2,dosp7mm,1510121623,0,None,Cornell,False,/r/Cornell/comments/78cb5y/alma_mater/dosp7xl/,,4.31024
78cb5y,1508806536,[removed],SwissWatchesOnly,None,78cb5y,0,None,1510471733,0,None,Cornell,False,/r/Cornell/comments/78cb5y/alma_mater/,,4.31024
dostp1k,1508811592,"Reuse, reduce, and recycle",SwissWatchesOnly,dosqnit,78cb5y,1,dosqnit,1510123856,0,None,Cornell,False,/r/Cornell/comments/78cb5y/alma_mater/dostp1k/,,4.31024


You can see above that utterances with the same speaker and conversation id have the same surprise as expected. Let's remove these duplicate entries so we can see more of the data.

In [16]:
most_surprising = transformed_corpus.get_utterances_dataframe().sort_values('meta.surprise', ascending=False).drop_duplicates(subset=['speaker', 'conversation_id']).head(10)
most_surprising

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,meta.surprise
id,,,,,,,,,,,,,,,
dbrix3s,1483037010,BILL BILL BILL BILL BILL BILL BILL BILL BILL B...,Straight_Derpin,dbqkxgp,5kst5l,22,dbqkxgp,1484130862,0,None,Cornell,False,,CS 2020,5.08975
diujudc,1497369546,Fall 2011 Admissions Stats:\n\nSchool | Apps |...,Dr_Narwhal,6h08sg,6h08sg,9,diujudc,1499312897,0,None,Cornell,False,,Physics &amp; Mathematics 2019,4.56458
cbnqq4d,1376508444,Even better is Tracfone's StraightTalk. They'r...,arandomaltaccount,cbnjwiu,1kbz4p,1,cbnjwiu,1429829700,0,None,Cornell,False,,,4.37768
c5sqkzc,1344908188,Fuck YAF. And fuck you. Don't let that hate ...,omgdonerkebab,y6cd9,y6cd9,0,c5sqkzc,1429631413,0,None,Cornell,False,,,4.34288
d64vv88,1470366925,"Eh, I would be hesitant to wear those. From th...",t3hasiangod,4w8d7e,4w8d7e,5,d64vv88,1473230691,0,None,Cornell,False,,,4.32161
dosp7xl,1508806551,"Far above Cayuga's waters,\nwith its waves of ...",SwissWatchesOnly,dosp7mm,78cb5y,-2,dosp7mm,1510121623,0,None,Cornell,False,/r/Cornell/comments/78cb5y/alma_mater/dosp7xl/,,4.31024
d5pb24c,1469403847,"They should be fine, though there are other op...",t3hasiangod,4ufm6z,4ufm6z,3,d5pb24c,1471654647,0,None,Cornell,False,,,4.27411
deydmud,1489576091,"Ah yes, because Newt is so tolerant of those w...",Bigmouthstrikesback,dexqq3j,5z98fa,3,dewdp2k,1491495064,0,None,Cornell,False,,,4.20492
crd3it7,1431968562,I doubt Tompkins was dry in 1940. [This articl...,howlingchief,crd3d1z,36ceei,2,crd2smi,1433132024,0,None,Cornell,False,,,4.12254


Now, let's look at some of the least surprising entries.

In [17]:
least_surprising = transformed_corpus.get_utterances_dataframe().sort_values('meta.surprise').drop_duplicates(subset=['speaker', 'conversation_id']).head(10)
least_surprising

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,meta.surprise
id,,,,,,,,,,,,,,,
cylyz8l,1451942801,I don't believe there is a minimum GPA require...,cryptkeep,3zgnom,3zgnom,7,cylyz8l,1454279614,0,None,Cornell,False,,,2.54516
dvir3qu,1520758241,CHEM 2150 is not a class you want to take if y...,rwaterbender,83daa2,83daa2,2,dvir3qu,1525042213,0,None,Cornell,False,/r/Cornell/comments/83daa2/chem_2150_for_an_ae...,AEP '20,2.57796
e7wd72e,1539728814,a) You can't minor in AEP (but you can minor i...,rwaterbender,9ora8i,9ora8i,5,e7wd72e,1541128816,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",Cornell,False,/r/Cornell/comments/9ora8i/conflicting_time_sc...,AEP '20,2.60018
d4itxwh,1466559788,The placement exams are generally final exams ...,laveritecestla,4p84yz,4p84yz,1,d4itxwh,1469205803,0,None,Cornell,False,,,2.64712
e3b5chg,1532980787,All of the other responses here are spot-on bu...,Fencerman2,92rvm5,92rvm5,4,e3b5chg,1536885658,0,None,Cornell,False,/r/Cornell/comments/92rvm5/what_to_expect_for_...,COE | CS '20,2.65595
dkfmynl,1500482307,"Of course you can. Having said that, you may h...",laveritecestla,6o9mzn,6o9mzn,11,dkfmynl,1501384309,0,None,Cornell,False,,Biomedical Engineering '18,2.65738
e7y72uc,1539800251,"Everything requires 2110 as a prereq, except 2...",rwaterbender,e7y6rt5,9oz71e,2,e7y5e4h,1541159585,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",Cornell,False,/r/Cornell/comments/9oz71e/help_guiding_a_lost...,AEP '20,2.65856
din8ic4,1496955479,"Imo, the best way to boost your chances is to ...",Fencerman2,6g3p38,6g3p38,4,din8ic4,1499182410,0,None,Cornell,False,,CS '20,2.68509
dn019j5,1505406483,Do you think you could determine the root of t...,SantaSoul,dmxd34b,6zqlyk,1,dmxcm3z,1506775045,0,None,Cornell,False,,CS/Stats 2020,2.69324


### Comparison to SpeakerConvoDiversity

In [18]:
from convokit import SpeakerConvoDiversity

scd = SpeakerConvoDiversity('div', select_fn=lambda df, row, aux: (df.convo_id != row.convo_id) & (df.speaker == row.speaker), speaker_cols=['n_convos'], aux_input={'n_iters': 50, 'cmp_sample_size': 100, 'ref_sample_size': 100}, verbosity=100)

In [19]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# for utt in corpus.iter_utterances():
#     tokens = cv.build_analyzer()(utt.text)
#     toks = [{'toks': [{'tok': x} for x in tokens]}]
#     utt.add_meta('tokens', toks)

# corpus.get_utterances_dataframe()

In [20]:
from convokit.text_processing import TextParser

tokenizer = TextParser(mode='tokenize', output_field='tokens', verbosity=1000)
subset_corpus = tokenizer.transform(subset_corpus)

1000/20700 utterances processed
2000/20700 utterances processed
3000/20700 utterances processed
4000/20700 utterances processed
5000/20700 utterances processed
6000/20700 utterances processed
7000/20700 utterances processed
8000/20700 utterances processed
9000/20700 utterances processed
10000/20700 utterances processed
11000/20700 utterances processed
12000/20700 utterances processed
13000/20700 utterances processed
14000/20700 utterances processed
15000/20700 utterances processed
16000/20700 utterances processed
17000/20700 utterances processed
18000/20700 utterances processed
19000/20700 utterances processed
20000/20700 utterances processed
20700/20700 utterances processed


In [21]:
div_transformed = scd.transform(subset_corpus)

joining tokens across conversation utterances
100 / 15394
200 / 15394
300 / 15394
400 / 15394
500 / 15394
600 / 15394
700 / 15394
800 / 15394
900 / 15394
1000 / 15394
1100 / 15394
1200 / 15394
1300 / 15394
1400 / 15394
1500 / 15394
1600 / 15394
1700 / 15394
1800 / 15394
1900 / 15394
2000 / 15394
2100 / 15394
2200 / 15394
2300 / 15394
2400 / 15394
2500 / 15394
2600 / 15394
2700 / 15394
2800 / 15394
2900 / 15394
3000 / 15394
3100 / 15394
3200 / 15394
3300 / 15394
3400 / 15394
3500 / 15394
3600 / 15394
3700 / 15394
3800 / 15394
3900 / 15394
4000 / 15394
4100 / 15394
4200 / 15394
4300 / 15394
4400 / 15394
4500 / 15394
4600 / 15394
4700 / 15394
4800 / 15394
4900 / 15394
5000 / 15394
5100 / 15394
5200 / 15394
5300 / 15394
5400 / 15394
5500 / 15394
5600 / 15394
5700 / 15394
5800 / 15394
5900 / 15394
6000 / 15394
6100 / 15394
6200 / 15394
6300 / 15394
6400 / 15394
6500 / 15394
6600 / 15394
6700 / 15394
6800 / 15394
6900 / 15394
7000 / 15394
7100 / 15394
7200 / 15394
7300 / 15394
7400 / 15394
7

Here are the speaker convo entries that have the highest diversity score.

In [22]:
div_transformed.get_speaker_convo_attribute_table(attrs=['div']).sort_values('div', ascending=False).head(10)

,speaker,convo_id,convo_idx,div
id,,,,
Straight_Derpin__5kst5l,Straight_Derpin,5kst5l,34,4.587125
Dr_Narwhal__6h08sg,Dr_Narwhal,6h08sg,75,4.537792
rrrrrrr1131__8l3xht,rrrrrrr1131,8l3xht,25,4.505238
ScottVandeberg__8tlcdl,ScottVandeberg,8tlcdl,81,4.497166
SwissWatchesOnly__9hcpip,SwissWatchesOnly,9hcpip,129,4.495047
t3hasiangod__5v6sqb,t3hasiangod,5v6sqb,590,4.492316
sasha07974__8v40c1,sasha07974,8v40c1,42,4.489417
t3hasiangod__4ufm6z,t3hasiangod,4ufm6z,262,4.482046
Straight_Derpin__7twbgg,Straight_Derpin,7twbgg,140,4.476163


Notice that the diversity scores returned by `SpeakerConvoDiversity` are slightly different from the scores returned by the `Surprise` transformer. This difference can be attributed to the addition of Laplace smoothing in the `Surprise` transformer to account for out of vocabulary tokens. The `SpeakerConvoDiversity` transformer deals with OOV tokens by simply treating their count as 1. If you run the `Surprise` transformer with the `smooth` flag set to false, the transformer will treat OOV tokens the same way `SpeakerConvoDiversity` does. When run without smoothing, the `Surprise` transformer returns the same scores as `SpeakerConvoDiversity`.

Here are the least diverse speaker-convo entries based on the SpeakerConvoDiversity transformer.

In [23]:
div_transformed.get_speaker_convo_attribute_table(attrs=['div']).sort_values('div').head(10)

,speaker,convo_id,convo_idx,div
id,,,,
laveritecestla__4pylgl,laveritecestla,4pylgl,74,4.206550
happysted__8qssd1,happysted,8qssd1,69,4.214580
t3hasiangod__3wtoeo,t3hasiangod,3wtoeo,46,4.220175
Fencerman2__6tiomd,Fencerman2,6tiomd,111,4.223014
Pjcrafty__5apodz,Pjcrafty,5apodz,17,4.224754
shadowclan98__9mfj81,shadowclan98,9mfj81,97,4.227928
shadowclan98__6njs8z,shadowclan98,6njs8z,8,4.228818
Fencerman2__90r1nf,Fencerman2,90r1nf,231,4.231099
dedicateddan__1bdpdq,dedicateddan,1bdpdq,7,4.232478
